In [1]:
import sys
import time
import torch
import torch.nn.functional as F
from utils import load_data, mat2tensor, regression_loss
from model import GCN, GAT
import numpy as np
import dgl

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device

device(type='cpu')

In [6]:
features_list, adjM, labels, train_val_test_idx, dl = load_data('iYO844')
features_list = [mat2tensor(features).to(device) for features in features_list]
features_list

[tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7155,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7173,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7082,  ..., 0.0000, 0.0000, 0.0000]]),
 tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0733, 0.0126, 0.0197],
         [0.0000, 0.0000, 0.0000,  ..., 0.0750, 0.0100, 0.0320],
         [0.0000, 0.0000, 0.0000,  ..., 0.0647, 0.0043, 0.0148],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0851, 0.0036, 0.0370],
         [0.0000, 0.0000, 0.0000,  ..., 0.0644, 0.0120, 0.0240],
         [0.0000, 0.0000, 0.0000,  ..., 0.0771, 0.0129, 0.0293]])]

In [4]:
labels

array([[2.43000000e-02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.72000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00],
       [0.000000

In [15]:
labels_tensor = mat2tensor(labels).to(device)
labels_tensor

tensor([[2.4300e-02, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.000

In [17]:
in_dims = [features.shape[1] for features in features_list]
in_dims

[384, 384]

In [18]:
## Set train, val, test index
labels = torch.FloatTensor(labels).to(device)
train_idx = train_val_test_idx['train_idx']
train_idx = np.sort(train_idx)
val_idx = train_val_test_idx['val_idx']
val_idx = np.sort(val_idx)
test_idx = train_val_test_idx['test_idx']
test_idx = np.sort(test_idx)

In [19]:
## Build graph
g = dgl.from_scipy(adjM+(adjM.T))
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)
g = g.to(device)

# Set model
num_labels = dl.labels_train['num_labels']

In [39]:
net = GCN(g, in_dims, 64, num_labels, 1, F.elu, 0.5)
net.to(device)

GCN(
  (layers): ModuleList(
    (0): GraphConv(in=64, out=64, normalization=both, activation=<function elu at 0x7f8ca786faf0>)
    (1): GraphConv(in=64, out=2, normalization=both, activation=None)
  )
  (fc_list): ModuleList(
    (0): Linear(in_features=384, out_features=64, bias=True)
    (1): Linear(in_features=384, out_features=64, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [21]:
logits = net(features_list)
print(logits[train_idx].shape)
logits[train_idx]

torch.Size([44, 2])


tensor([[-0.1145,  0.6282],
        [-0.3118,  0.3810],
        [-0.3781, -0.6077],
        [-0.7508,  0.2979],
        [-0.2201,  1.2604],
        [ 0.0720,  0.1400],
        [ 0.4754,  1.6644],
        [ 0.1122,  0.5057],
        [-0.0907,  0.5406],
        [-0.9749,  0.5171],
        [-0.6224,  0.2868],
        [ 0.1320, -0.0150],
        [-0.4131,  0.5809],
        [ 0.3016,  0.1656],
        [-0.0804,  0.0492],
        [-0.1920,  0.5104],
        [-0.5039,  0.6995],
        [-0.6519, -0.4109],
        [-0.3404, -0.1025],
        [-0.4442,  0.8889],
        [-0.3333,  0.1513],
        [-0.5208, -0.0598],
        [ 0.3690,  0.1578],
        [-0.2350,  1.0697],
        [ 0.0280,  0.6063],
        [-0.4153,  0.0568],
        [ 0.1604,  0.3436],
        [ 0.7139,  0.1902],
        [ 0.2062,  0.1504],
        [-1.0455, -0.0944],
        [-0.1810, -0.3948],
        [ 0.2693,  0.0943],
        [ 0.3520, -0.0388],
        [-0.5271, -0.0930],
        [-0.2810,  0.8152],
        [-0.2156,  0

In [22]:
print(labels[train_idx].shape)
labels[train_idx]

torch.Size([44, 2])


tensor([[2.4300e-02, 0.0000e+00],
        [0.0000e+00, 1.7200e+00],
        [3.0000e+01, 0.0000e+00],
        [0.0000e+00, 6.4250e+00],
        [2.4075e+01, 1.1900e+03],
        [0.0000e+00, 1.1000e+00],
        [8.0000e-03, 3.0000e-01],
        [1.0000e-03, 7.0000e-03],
        [5.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.2000e-02, 8.3000e+01],
        [8.9000e-03, 0.0000e+00],
        [5.0000e-02, 0.0000e+00],
        [0.0000e+00, 5.3000e+00],
        [0.0000e+00, 2.5950e-01],
        [2.1000e+00, 0.0000e+00],
        [1.3500e+01, 6.0000e+01],
        [8.0000e+00, 0.0000e+00],
        [4.2880e-01, 4.9657e+03],
        [8.7000e+00, 0.0000e+00],
        [0.0000e+00, 1.1000e+01],
        [2.0200e+00, 6.2600e+00],
        [5.9000e-02, 0.0000e+00],
        [3.3000e-02, 0.0000e+00],
        [4.0250e-02, 2.5775e+02],
        [5.0000e+00, 4.8500e+01],
        [1.4000e-01, 0.0000e+00],
        [3.0000e+00, 0.0000e+00],
        [3.3000e-04, 4.6000e+00],
        [0.000

In [23]:
# compute the regression loss
def regression_loss(logits, labels):
    return F.mse_loss(logits, labels)

In [24]:
train_loss = regression_loss(logits[train_idx], labels[train_idx])

In [25]:
train_loss

tensor(298309.9688, grad_fn=<MseLossBackward0>)

In [28]:
with torch.no_grad():
    logits = net(features_list)
    test_logits = logits[test_idx]
    test_logits.shape
    pred = test_logits.cpu().numpy()
pred.shape

(24, 2)

In [29]:
y_true = dl.labels_test['data'][dl.labels_test['mask']]
y_true.shape

(24, 2)

In [30]:
from sklearn.metrics import mean_squared_error

In [38]:
km_error =  mean_squared_error(y_true.T[0], pred.T[0])
print(km_error)

kcat_error =  mean_squared_error(y_true.T[1], pred.T[1])
print(kcat_error)

151.64680787172406
1087641.795832917


In [37]:
km_error

151.64680787172406

In [35]:
pred.T

array([[-0.08458439, -0.3827634 , -0.35651696, -0.12638767, -0.03449817,
        -0.05850934, -0.27367902,  0.31351304, -0.10706603, -0.5453299 ,
        -0.6696881 ,  0.2398427 , -0.00914084, -0.088196  , -0.2677754 ,
         0.05398614, -0.33292863,  0.6396899 , -0.8642582 ,  0.5878214 ,
        -0.5064774 , -0.44223532, -0.07550129, -0.78081936],
       [ 0.30699763, -0.42385587, -0.32325754, -0.14331095,  0.19843191,
         1.7009141 , -1.0412201 ,  0.4743387 ,  0.84876823,  0.05098727,
         0.37899613,  0.6626741 ,  0.43339   ,  0.3884286 , -0.44346988,
         0.2360711 ,  0.16602506,  1.264272  ,  0.19299929,  1.7473607 ,
         0.70039725,  0.02379687,  0.7557141 , -0.67481166]],
      dtype=float32)